En los ejercicios para realizar de clase 3, vimos el dataset winequality-red.csv, el cual consiste en datos de vinos rojos basados en datos físico-químicos y una metrica de calidad de vino. Más info en Kaggle. Usando el mismo dataset y basando en el estudio de variables anteriormente realizado, eligiendo algún método de ensamble visto de clase

In [1]:
# Importa las librerias necesarias
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Carga la ruta al archivo de datos
ruta = os.path.dirname((os.path.abspath('Ejercicio_Resuelto')))
ruta_datos = os.path.join(ruta, "datasets/winequality-red_procesados.csv")

# Lectura del archivo a DataFrame
wines = pd.read_csv(ruta_datos)

In [3]:
# Busca informacion del dataset
wines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1255 entries, 0 to 1254
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1255 non-null   float64
 1   volatile acidity      1255 non-null   float64
 2   residual sugar        1255 non-null   float64
 3   chlorides             1255 non-null   float64
 4   total sulfur dioxide  1255 non-null   float64
 5   sulphates             1255 non-null   float64
 6   alcohol               1255 non-null   float64
 7   quality               1255 non-null   int64  
dtypes: float64(7), int64(1)
memory usage: 78.6 KB


In [4]:
# Muestra los primeros registros
wines.head()

,fixed acidity,volatile acidity,residual sugar,chlorides,total sulfur dioxide,sulphates,alcohol,quality
0,-0.527990,1.022750,-0.577260,-0.245629,-0.361750,-0.631289,-0.990401,5
1,-0.287516,2.067774,0.259784,0.717151,0.724247,0.290757,-0.610922,5
2,-0.287516,1.371091,-0.098949,0.454575,0.296430,0.060245,-0.610922,5
3,1.756519,-1.415640,-0.577260,-0.289392,0.493884,-0.477615,-0.610922,6
4,-0.527990,0.790522,-0.696838,-0.289392,-0.164296,-0.631289,-0.990401,5


1 - Construya un módelo de regresión. Juegue con las variables usando algún método de búsqueda de hiper-parámetros si hace falta.

In [5]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [6]:
# Separa las caracteristicas y el objetivo
X = wines.drop('quality', axis=1)
y = wines['quality']

In [7]:
# Divide los datos en conjunto de entrenamiento y prueba 80/20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Normaliza el set de entrenamiento y de prueba de las caracteristicas
sc_X = StandardScaler()

X_train_scaled = sc_X.fit_transform(X_train) 
X_test_scaled = sc_X.transform(X_test)

In [9]:
# Define el modelo de regresion lineal
model = LinearRegression()

In [10]:
# Define la grilla de hiper-parametros
param_grid = {
    'fit_intercept': [True, False],  # Si se incluye el intercepto o no
}

In [11]:
# Configura la busqueda de hiper-parametros por grilla con validacion cruzada
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')

In [12]:
# Entrena el modelo
grid_search.fit(X_train_scaled, y_train)


GridSearchCV(cv=5, estimator=LinearRegression(),
             param_grid={'fit_intercept': [True, False]},
             scoring='neg_mean_squared_error')

In [13]:
# Obtiene los mejores hiper-parametros
best_params = grid_search.best_params_
print(f"Mejores hiperparámetros: {best_params}")


Mejores hiperparámetros: {'fit_intercept': True}


In [14]:
# Obtiene el MSE en el conjunto de prueba
y_pred = grid_search.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error en el conjunto de prueba: {mse}")

Mean Squared Error en el conjunto de prueba: 0.403027405132609


2 - Usando la métrica de evaluación del modelo del SVM y el arbol, compare los resultados obtenidos con respecto a los anteriormente.

Modelo TreeRegressor:

In [15]:
# Importa la clase del modelo del arbol de regresion
from sklearn.tree import DecisionTreeRegressor

# Crea el modelo de arbol de regresion
tree_regressor = DecisionTreeRegressor()

In [16]:
# Define la grilla de hiperparametros
param_grid = [
    {"criterion": ["squared_error"],
     "splitter": ["best"],
     "max_depth": [x for x in range(10)],
     "min_samples_split": [x for x in range(20)],
     "min_samples_leaf": [x for x in range(10)],
     "max_features": [None],
     "ccp_alpha": [x * 0.01 for x in range(0, 50)]}
]

In [17]:
# Configura GridSearchCV
grid_search = GridSearchCV(tree_regressor, param_grid, cv=5, scoring='neg_mean_squared_error', refit=True)

In [18]:
# Entrena el modelo
grid_search.fit(X_train_scaled, y_train)

c:\Users\fabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
135500 fits failed out of a total of 500000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50000 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\fabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\fabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Users\fabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\skle

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid=[{'ccp_alpha': [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06,
                                        0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13,
                                        0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2,
                                        0.21, 0.22, 0.23, 0.24, 0.25, 0.26,
                                        0.27, 0.28, 0.29, ...],
                          'criterion': ['squared_error'],
                          'max_depth': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                          'max_features': [None],
                          'min_samples_leaf': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                          'min_samples_split': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                10, 11, 12, 13, 14, 15, 16, 17,
                                                18, 19],
                          'splitter': ['best']}],
             scoring='neg_mean_squared_error')

In [19]:
# Obtiene los mejores hiper-parametros
best_params = grid_search.best_params_
print(f"Mejores hiperparámetros: {best_params}")

Mejores hiperparámetros: {'ccp_alpha': 0.01, 'criterion': 'squared_error', 'max_depth': 4, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'best'}


In [20]:
# Obtiene el MSE en el conjunto de prueba
y_pred = grid_search.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error en el conjunto de prueba: {mse}")

Mean Squared Error en el conjunto de prueba: 0.4277956855525221


Modelo SVM Regressor:

In [21]:
# Importa la clase del modelo SVM
from sklearn.svm import SVR

# Crea el modelo de arbol de regresion
svm = SVR()

In [22]:
# Configura GridSearchCV
grid_search = GridSearchCV(svm,
                    [{"C": [0.01, 0.1, 1, 5, 10, 100], "kernel": ["linear"]},
                     {"C": [0.01, 0.1, 1, 5, 10, 100], "gamma": [0.1, 0.5, 1, 2, 10, 100], "kernel": ["rbf", "sigmoid"]},
                     {"C": [0.01, 0.1, 1, 5, 10, 100], "degree": [2, 3, 4, 5, 6], "kernel": ["poly"]}],
                    refit=True,
                    cv=5,
                    scoring='neg_mean_squared_error')

In [23]:
# Entrena el modelo
grid_search.fit(X_train_scaled,y_train)

GridSearchCV(cv=5, estimator=SVR(),
             param_grid=[{'C': [0.01, 0.1, 1, 5, 10, 100],
                          'kernel': ['linear']},
                         {'C': [0.01, 0.1, 1, 5, 10, 100],
                          'gamma': [0.1, 0.5, 1, 2, 10, 100],
                          'kernel': ['rbf', 'sigmoid']},
                         {'C': [0.01, 0.1, 1, 5, 10, 100],
                          'degree': [2, 3, 4, 5, 6], 'kernel': ['poly']}],
             scoring='neg_mean_squared_error')

In [24]:
# Obtiene los mejores hiper-parametros
best_params = grid_search.best_params_
print(f"Mejores hiperparámetros: {best_params}")

Mejores hiperparámetros: {'C': 0.01, 'kernel': 'linear'}


In [25]:
# Obtiene el MSE en el conjunto de prueba
y_pred = grid_search.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error en el conjunto de prueba: {mse}")

Mean Squared Error en el conjunto de prueba: 0.40180612289583134


| Modelo    | MSE  |
|-----------|------|
| SVR Reg.  | 0.40 |
| Tree Reg. | 0.43 |
| L. Reg.   | 0.40 |

3 - ¿Cuál fue el mejor modelo en métricas? Y de facilidad de entender los resultados? Discuta los resultados.

Observaciones: Los 3 modelos dieron resultados similares, con el 'Tree Regressor' con un MSE ligeramente mayor a los otros dos modelos. Los 3 resultan fácil de interpretar.